<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/langchain_separatellms_vertexai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet --upgrade google-cloud-aiplatform langchain neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 18.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.0/321.0 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.0 MB/s eta 0:00:00


In [2]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
from langchain.graphs import Neo4jGraph

graph = Neo4jGraph(
    url="bolt://44.204.158.102:7687",
    username="neo4j",
    password="glance-machines-hardcopies"
)

In [2]:
from google.colab import files

uploaded = files.upload()

Saving credentials.json to credentials.json


In [3]:
import os

from langchain.chat_models import ChatVertexAI
from langchain.chains import GraphCypherQAChain

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "credentials.json"

In [8]:
from langchain.chains.llm import LLMChain
from langchain.chains.graph_qa.prompts import CYPHER_GENERATION_PROMPT, CYPHER_QA_PROMPT
from langchain.chains.llm import LLMChain


cypher_llm = ChatVertexAI(temperature=0, model_name="codechat-bison")
qa_llm = ChatVertexAI(temperature=0, model_name="chat-bison")

chain = GraphCypherQAChain(
    graph=graph,
    cypher_generation_chain=LLMChain(llm=cypher_llm, prompt=CYPHER_GENERATION_PROMPT),
    qa_chain=LLMChain(llm=qa_llm, prompt=CYPHER_QA_PROMPT),
    verbose=True)

In [9]:
chain.run("Who is the most followed fortnite streamer?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (s:Stream)-[:PLAYS]->(:Game {name: 'Fortnite'})
WITH s, s.followers AS followers
ORDER BY followers DESC
LIMIT 1
RETURN s.name AS name, followers

Full Context:
[{'name': 'thegrefg', 'followers': 7269018}]

> Finished chain.


' thegrefg is the most followed Fortnite streamer, with 7,269,018 followers. '